# Пакеты

In [ ]:
import math

from collections import defaultdict
from functools import lru_cache
from enum import Enum

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats


from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
%matplotlib inline


from pylab import rcParams

rcParams['figure.figsize'] = 18, 8


!pip install BeeHiveOptimization
from BeeHiveOptimization import Bees, Hive, BeeHive

!pip install geneticalgorithm
from geneticalgorithm import geneticalgorithm as ga # пакет с простым генетическим алгоритмом

# Идея алгоритма

In [ ]:
codes = [
    '00000072_000',
    '00000150_002',
    '00000181_061',
    '00002578_000',
    '00003400_003',
    '00001075_024'
]



for ind in codes:
    
    origin = Image.open(f'../input/digital-transformation-2020/Dataset/Dataset/Origin/{ind}.png')
    
    expert = Image.open(f'../input/digital-transformation-2020/Dataset/Dataset/Expert/{ind}_expert.png')

    model1 = Image.open(f'../input/digital-transformation-2020/Dataset/Dataset/sample_1/{ind}_s1.png')

    model2 = Image.open(f'../input/digital-transformation-2020/Dataset/Dataset/sample_2/{ind}_s2.png')
    
    model3 = Image.open(f'../input/digital-transformation-2020/Dataset/Dataset/sample_3/{ind}_s3.png')
    
    fig, axes = plt.subplots(1, 5)

    
    axes[0].imshow(np.asarray(origin))
    axes[0].set_title(f"Оригинальный снимок")
    
    axes[1].imshow(np.asarray(expert))
    axes[1].set_title(f"Маска эксперта")
    
    axes[2].imshow(np.asarray(model1))
    axes[2].set_title(f'Маска модели 1')

    axes[3].imshow(np.asarray(model2))
    axes[3].set_title(f'Маска модели 2')
    
    axes[4].imshow(np.asarray(model3))
    axes[4].set_title(f'Маска модели 3')

    fig.set_figwidth(18)    #  ширина и
    fig.set_figheight(10)    #  высота "Figure"

    plt.show()

В файле *"../input/digital-transformation-2020/Dataset/Dataset/DX_TEST_RESULT_FULL.csv"* хранятся данные, которые мы сводим к следующему: **для каждого снимка некоторая модель выдает один набор фигур, а эксперт -- другой, причем этот набор фигур может быть и пустым**. Отличие модельной конфигурации от экспертной эксперт субъективно определяет по шкале от 1 до 5 (чем больше, тем лучше).

Алгоритм заключается в следующем: **сходство двух изображений определяется количеством штрафа, наложенного за разные необходимые трансформации фигур одного изображения в фигуры другого**.

Штрафы назначаются за следующие действия:

* если требуется создать (недостающую) фигуру, которая есть у эксперта и которой нет у модели, налается штраф, пропорциональный площади этой фигуры
* если требуется удалить (лишнюю) фигуру, которой нет у эксперта, но которая есть у модели, налается штраф, пропорциональный площади этой фигуры
* если нужно преобразовать эллипс или наоборот (у модели эллипс, а у эксперта там -- прямоугольник), налагается свой штраф, связанный с площадями
* если требуется переместить фигуру модели на некоторое расстояние, чтобы центры фигур модели и эксперта совпали, налается штраф, пропорциональный расстоянию
* если требуется увеличить/уменьшить фигуру модели по вертикали/горизонтали (чтоб получить фигуру эксперта), за каждое такое изменение налается свой штраф
* кроме этого, налагается штраф, пропорциональный числу несоответствующих (непарных) фигур между конфигурациями модели и эксперта (если у эксперта есть только одна фигура, а модель нашла 10, это нехорошо)

Как определятся, что две разные фигуры у модели и эксперта на самом деле идентичны и должны сводиться одна в другую? Для этого есть параметр `tresh`. Если расстояние между центрами фигур меньше этого параметра, они будут идентичны. При этом пара идентичных фигур выбирается как пара фигур с минимальным расстоянием, меньшим `tresh`.

Таким образом, **для каждой пары изображений мы можем вычислить требуемый набор трансформаций, за каждую трансформацию наложить свой штраф, а сумма штрафов будет выражать степень схожести изображений**.

Сами штрафы за каждую трансформацию подбираются эволюционным алгоритмом. Цель алгоритма при подборе штрафов -- минимизировать корреляцию Спирмена (устремить к -1) между получаемыми суммами штрафов (в неизвестно каких диапазонах) и целевыми оценками (в диапазоне от 1 до 5). После этого, для преведения наших оценок к целевому диапазону используется простая линейная регрессия с трансформацией данных. 

# Нужные классы

In [ ]:
class Shape(Enum):
    circle = 0
    rect = 1


class Point:
    """
    точка на плоскости
    """
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    @staticmethod    
    def dist(A, B):
        return math.hypot(A.x - B.x, A.y - B.y)

class Figure:
    """
    фигура (не имеет особого прям значения, эллипс это или прямоугольник)
    """
    def __init__(self, center, horizontal, vertical, shape):
        self.center = center
        self.hor = horizontal
        self.ver = vertical
        
        self.shape = Shape.circle if shape == 'circle' else Shape.rect
        
        self.S = 4 * self.hor * self.ver if self.shape == Shape.rect else math.pi * self.hor * self.ver # площадь фигуры
        
    def S_diff(self): # разность площади между прямоугольником и эллипсом
        return self.hor * self.ver * (4 - math.pi)
    
    @staticmethod
    def dist(A, B):
        """
        расстояние между центрами фигур
        """
        return Point.dist(A.center, B.center)
    
    
    @staticmethod
    def transforms(model, expert): # считает наказания за трансформации
        """
        возвращает массив наказаний за увеличение/уменьшение фигуры model по вертикали/горизонтали так, чтобы сводить ее в фигуру expert
        
        формула наказания -- это большее/меньшее * на размер у expert
        
        таким образом, если нужно уменьшить вертикаль в два раза с 10 до 5, это не так страшно, как уменьшать вертикаль в 2 раза с 100 до 50
        
        больше-меньше/вертикаль-горизонталь выводятся отдельно, чтобы за них была отдельная ошибка
        """
                
        if model.ver > expert.ver: # если надо уменьшать вертикаль
            ver_up, ver_down = 0, model.ver #/expert.ver 
        elif model.ver < expert.ver and model.ver > 0: # если надо увеличивать вертикаль
            ver_up, ver_down = expert.ver**2 / model.ver, 0
        else:
            ver_up, ver_down = 0, 0
        
        
        if model.hor > expert.hor: # если надо уменьшать горизонталь
            hor_up, hor_down = 0, model.hor #/expert.hor 
        elif model.hor < expert.hor and model.hor > 0:
            hor_up, hor_down = expert.hor**2 / model.hor, 0
        else:
            hor_up, hor_down = 0, 0
        
        return np.array([ver_up, ver_down, hor_up, hor_down])
        

        

# Чтение и форматирование данных

## Таблица с целями

Целевая таблица:

In [ ]:
target_frame_full = pd.read_csv('../input/digital-transformation-2020/Dataset/Dataset/OpenPart.csv')

target_frame_full

Следующий код преобразует таблицу таким образом, чтобы остался только столбец уникальных имён и столбец цели:

In [ ]:
def table_preparation(table):
    image_names = table['Case'].values

    def get_sample_name(sample_number):
        return [f'{code[0]}_s{sample_number}.png' for code in (t.split('.') for t in image_names)]

    samples_names = get_sample_name(1) + get_sample_name(2) + get_sample_name(3)
    
    target_frame = pd.DataFrame({
        'names': samples_names,
        'target': np.concatenate([table.iloc[:,1].values, table.iloc[:,2].values, table.iloc[:,3].values])
    })

    return target_frame
    

In [ ]:
target_frame_full = table_preparation(target_frame_full)

target_frame_full

## Чтение и трансформация выборки

In [ ]:
fig_df = pd.read_csv('../input/digital-transformation-2020/Dataset/Dataset/DX_TEST_RESULT_FULL.csv').iloc[:,:-1].rename(columns=lambda x: x.strip()) # последний столбец пустой, имена содержат лишние пробелы

fig_df

Далее я добавляю к таблице столбец `name`, который является ключом для соединения с `target_frame`

In [ ]:
fig_df['user_name'].drop_duplicates() # имена доступных сущностей

In [ ]:
fig_samples = fig_df.query("user_name != 'Expert'")

new_name = [f"{file}_s{s[-1]}.png" for file, s in zip (fig_samples['file_name'], fig_samples['user_name'])]

fig_samples['name'] = new_name

fig_samples

Аналогичная таблица у эксперта (но здесь уточнять имя уже не нужно):

In [ ]:
fig_expert = fig_df.query("user_name == 'Expert'").drop('user_name', 1)
fig_expert

## Перегон в словари

Теперь каждому изображению (из масок моделей и эксперта) нужно поставить в соотвествие список входящих в него сущностей, ибо дальше работа ведётся именно с этими сущностями.

In [ ]:
samples_dict = defaultdict(list)

for name, x, y, h, v, s in zip(fig_samples['name'], fig_samples['xcenter'], fig_samples['ycenter'], fig_samples['rhorizontal'], fig_samples['rvertical'], fig_samples['shape']):
    p = Point(x,y)
    samples_dict[name].append(Figure(p, h, v, s))


In [ ]:
expert_dict = defaultdict(list)

for name, x, y, h, v, s in zip(fig_expert['file_name'], fig_expert['xcenter'], fig_expert['ycenter'], fig_expert['rhorizontal'], fig_expert['rvertical'], fig_expert['shape']):
    p = Point(x,y)
    expert_dict[name].append(Figure(p, h, v, s))

Эта функция обрезает конец имени изображения модели, чтобы по нему можно было взять объекты эксперта

In [ ]:
def remove_end(string):
    """
    преобразует 12345_000_s1.png в 12345_000
    """
    t = string.split('_')
    return f'{t[0]}_{t[1]}'

## Удаление лишних наблюдений

Под "лишними" подразумеваются пары снимков модель-образец, у которых либо нет объектов на обоих снимках (тогда это категория 5, потому что модель не нашла ничего на плохом снимке), либо нет объектов только на одном из снимков (тогда это категория 1, потому что модель либо вообще не нашла нужное, либо нашла что-то на идеальном снимке).

In [ ]:
bad_mask = [(len(samples_dict[name]) > 0) and (len(expert_dict[remove_end(name)]) > 0) for name in target_frame_full['names']]

target_frame = target_frame_full.iloc[np.array(bad_mask),:]

target_frame

# Реализация штрафов

Для каждой пары конфигураций модель-эксперт нужно произвести разбор их объектов, сравнения и т. п. Каждая необходимая трансформация имеет свою категорию и свои диапазоны значений.

В итоге разбора **создается словарь с суммами по категориям штрафов** (эти суммы, не зависимые от конкретных значений штрафа, будем называть наказаниями). Это нужно сделать и сохранить заранее с той целью, чтобы избежать повторяющихся разборов, то есть лишних накладных расходов. Кроме того, конкретные значения штрафов заранее неизвестны, их нужно подбирать; и удобнее всего будет просто умножать штрафы на сохраненные значения. 

Получаемый в итоге словарь имеет следующие элементы:

* `to_delete`: сумма площадей фигур, которые требуется удалить (удалить у модели, ибо у эксперта их нет, они лишние)
* `to_create`: сумма площадей фигур, которые требуется добавить (ибо модель их не обнаружила)
* `shape`: сумма особых площадей фигур, у которых нужно поменять форму (особая площадь -- это разность между площадью прямоугольника и площадью эллипса; здесь имеется в виду, что изменение формы должно чего-то стоить, но это не настолько страшно, как отсутствие нужной фигуры)
* `shift`: сумма расстояний, на которые нужно сдвинуть некоторые фигуры модели, чтобы получить фигуры эксперта,
* `ver_up`: сумма, связанная с наказанием за необходимость увеличивать вертикальную сторону (увеличивать у фигуры модели, чтобы совпало с фигурой эксперта); сами значения внутри суммы всегда равны большее/меньшее*(длину стороны у фигуры эксперта)
* `ver_down`: то же самое, только за уменьшение по вертикали
* `hor_up`: то же самое за увеличение по горизонтали
* `hor_down`: за уменьшение по горизонтали

Каждому из этих элементов будет соответствовать свой штраф.

Другие параметры алгоритма:

* `tresh`: максимальное расстояния между центрами фигур (фигуры у модели и фигуры у эксперта), чтобы считать эти фигуры идентичными (одна как бы должна свестись к другой, потому что покрывают они один и тот же объект)
* `coef`: итоговая сумма по штрафам выражается формулой

$$ s = s_{global} + over \cdot k_{over} + coef \cdot avg(s_{local})$$


где $s_{global}$ -- штрафы за добавление/удаление фигур, $k_{over}$ -- количество несоотвествующих фигур, $avg(s_{local})$ -- сумма штрафов за перемещения/изменения размеров, деленное на число фигур, которые в этом участвовали. `coef` меняет значимость этого слагаемого
* `over`: штраф за фигуры без соответствий


Следующая функция кеширует набор чисел, на которые нужно умножать штрафы, для каждой пары конфигураций модель-эксперт:

In [ ]:
@lru_cache(maxsize = 10000)
def cache_pairs(name_sample, name_exp, tresh):
    
    sample_arr = samples_dict[name_sample]
    expert_arr = expert_dict[name_exp]
    
    answer = defaultdict(float)
    
    ls = len(sample_arr)
    le = len(expert_arr)
    
    # следующий закомментрированный код раньше использовался, но теперь момент отсутствия фигур на минимум одном из изображений мы рассматриваем особо 
    
    #if ls == 0 and le == 0: # если модель и эксперт не находят патологий
    #    return answer, 0, 0
    
    #if ls > 0 and le == 0: # если эксперт ничего не нашел, а модель нашла, нужно наказать модель пропорционально площади лишних объектов
    #    answer['to_delete'] = sum((obj.S for obj in sample_arr))
    #    return answer, 0, ls
    
    #if ls == 0 and le > 0: # если модель ничего не нашла, а эксперт нашел, нужно наказать модель пропорционально площади несозданных объектов
    #    answer['to_create'] = sum((obj.S for obj in expert_arr))
    #    return answer, 0, le
    
    
    # матрица расстояний для пар объектов
    dist = np.empty((ls, le), dtype = np.float32)
    for x in range(ls):
        for y in range(le):
            dist[x, y] = Figure.dist(sample_arr[x], expert_arr[y])
    
    # дальше нужно определить все объекты с попарными расстоями не больше tresh
    dist_ravel = dist.ravel()
    
    s, e = [], [] # годные фигуры у каждого
    for i in np.argsort(dist_ravel): # начинаем искать пары по самым маленьким расстояниям в матрицу
        if dist_ravel[i] >= tresh:
            break
        
        si, se = (i) // le, (i+1) % le - 1 # перевод индексов из ravel в матричные
        #print(f's = {s} si = {si} i = {i} ravel = {dist_ravel}')
        s.append(sample_arr[si])
        e.append(expert_arr[se])
        
        answer['shift'] += dist_ravel[i] # суммирование штрафов за сдвиг
    
    
    # лишние объекты удаляются, недостающие объекты создаются 
    answer['to_delete'] = sum((obj.S for obj in sample_arr if obj not in s))
    answer['to_create'] = sum((obj.S for obj in expert_arr if obj not in e))    
    
    
    transforms = np.zeros(4)
    for model_result, expert_result in zip(s, e):
        transforms += Figure.transforms(model_result, expert_result) # по каждый паре идентичных объектов суммируем наказания
        
        if model_result.shape != expert_result.shape: # если надо поменять форму, за это тоже наказание
            answer['shape'] += expert_result.S_diff()
    
    
    answer['ver_up'] = transforms[0]
    answer['ver_down'] = transforms[1]
    answer['hor_up'] = transforms[2]
    answer['hor_down'] = transforms[3]
    
    return answer, len(s), math.fabs(ls - le) # последние два слагаемых удобнее выводить отдельно, не используя словарь

Эта функция возвращает результат для пары "снимков" и дополнительных параметров алгоритма (суммирует штрафы по полученным словарям):

In [ ]:
glob_set = set(['to_delete', 'to_create']) # категории, за которые штраф не усредняется

def result_function(sample_arr, expert_arr, name_sample, name_exp, tresh, params, coef = 5, over = 10): # name_sample, name_exp приходится использовать для хеширования, не хватило времени переписать код более чисто
    
    punishes, count, bad_count = cache_pairs(name_sample, name_exp, tresh)
    
    glob_sum = sum((params[key]*punishes[key] for key in glob_set)) # сумма по неусредняемым наказаниям
    
    if count > 0:
        sm = sum((params[key]*punishes[key] for key in params.keys() if key not in glob_set)) # сумма по усредняемым наказаниям
        return glob_sum + bad_count*over + sm/count*coef
    else:
        return glob_sum + bad_count*over

# Пример использования при случайных параметрах

## Создаем результирующую функцию с некоторыми фиксированными параметрами алгоритма 

Как видно, какие-то параметры мы храним в словаре, а какие-то легче передавать отдельно.

In [ ]:
params = {
    'to_delete': 2,
    'to_create': 10,
    'shape': 1,
    'shift': 8,
    'ver_up': 2,
    'ver_down': 2,
    'hor_up': 2,
    'hor_down': 2
}

def some_function(sample_arr, expert_arr, name_sample, name_exp):
    return result_function(sample_arr, expert_arr, name_sample, name_exp, tresh = 30, params = params, coef = 5, over = 1000)

## Результаты предсказания

In [ ]:
answer = [some_function(samples_dict[sample_arr], expert_dict[remove_end(sample_arr)], sample_arr, remove_end(sample_arr)) for sample_arr in target_frame['names']]

res = target_frame.copy()
res['predicted'] = answer

res

Пока что цель лишь в том, чтобы делать предсказания с сохранением порядка (то есть самые маленькие значения должны уходить в группу 5, самые большие, у которых максимальные штрафы, в 1). Следующая функция реализует простую метрику оценки качества предыдущего результата

In [ ]:
def get_corr(frame):
    """
    корреляция Спирмена
    
    поскольку цель инвертирована, эту корреляцию нужно минимизировать, устремляя к -1
    """
    
    return stats.spearmanr(frame['target'].values, frame['predicted'].values).correlation # корреляция Спирмена, чтобы сохранять порядок

get_corr(res)

Посмотрим несколько примеров масок с оценкой эксперта и нашим штрафом:

In [ ]:
for ind in (0, 1, 2, 3, 4, 5, 6, 7):
    
    sample_arr = target_frame_full['names'][ind]
    answer = some_function(samples_dict[sample_arr], expert_dict[remove_end(sample_arr)], sample_arr, remove_end(sample_arr))

    expert = Image.open(f'../input/digital-transformation-2020/Dataset/Dataset/Expert/{remove_end(sample_arr)}_expert.png')

    model = Image.open(f'../input/digital-transformation-2020/Dataset/Dataset/sample_1/{sample_arr}')

    fig, axes = plt.subplots(1, 2)

    axes[0].imshow(np.asarray(model))
    axes[0].set_title(f'Маска модели, штраф {answer}')

    axes[1].imshow(np.asarray(expert))
    axes[1].set_title(f"Маска эксперта, оценка {target_frame_full.query('names == @sample_arr')['target'].values[0]}")

    fig.set_figwidth(18)    #  ширина и
    fig.set_figheight(10)    #  высота "Figure"

    plt.show()

Из первого графика можно сделать вывод, что обнаружение объектов на идеальном снимке или их необнаружение на снимке с патологией сразу означает 1.

# Эволюционный алгоритм

Оптимальные параметры будем искать **методом роя частиц** (реализованным мной и хранящимся https://github.com/PasaOpasen/BeehiveMethod).

Сперва мы пользовались простым генетическим алгоритмом (источник: https://github.com/rmsolgi/geneticalgorithm), но оказалось, что метод роя частиц дает такие же результаты, но сходится в несколько раз быстрее. На всякий случай прежний метод поиска генетическим алгоритмом сохранён, его можно опробовать.

In [ ]:
def evaluate(df, params):
    """
    функция, которая датафрейму и параметрам алгоритма сопоставляет ошибку на этом датафрейме
    """
    
    # некоторые параметры храним в словаре, некоторые используем отдельно
    pr = {
        'to_delete': params[2],
        'to_create': params[3],
        'shape': params[4],
        'shift': params[5],
        'ver_up': params[6],
        'ver_down': params[7],
        'hor_up': params[8],
        'hor_down': params[9]
    }
    
    def some_function(sample_arr, expert_arr, name_sample, name_exp):
        return result_function(sample_arr, expert_arr, name_sample, name_exp, tresh = params[0], params = pr, coef = params[1], over = params[10])
    
    answer = [some_function(samples_dict[sample_arr], expert_dict[remove_end(sample_arr)], sample_arr, remove_end(sample_arr)) for sample_arr in df['names']]

    res = df.copy()
    res['predicted'] = answer

    return get_corr(res)
    

In [ ]:
def find_solution_ga(df):
    """
    поиск оптимальных параметров генетическим алгоритмом
    """
    
    def f(X):
        return evaluate(df, X)
    
    # границы по каждому параметру
    varbound = np.array([
        [0, 500], # tresh
        [0, 20], # coef
        [0, 2000], # to_delete
        [0, 1000], # to_create
        [0, 20], # shape
        [0, 1000], # shift
        [0, 50], # ver_up
        [0, 300], # ver_down
        [0, 50], # hor_up
        [0, 150], # hor_down
        [0, 10000] #over
    ])
    
    param = {
        'max_num_iteration': 400, 
        'population_size': 500, 
        'mutation_probability': 0.15, 
        'elit_ratio': 0.05, 
        'crossover_probability': 0.5, 
        'parents_portion': 0.3, 
        'crossover_type': 'two_point', 
        'max_iteration_without_improv': 50
    }


    model = ga(function = f, dimension = 11, variable_type = 'real', variable_boundaries = varbound, algorithm_parameters = param)

    model.run()
    
    
    return model.output_dict['variable']
    

In [ ]:
def find_solution(df):
    """
    поиск оптимальных параметров методом роя частиц
    """
    
    def f(X):
        return evaluate(df, X)
    
    # границы по каждому параметру
    varbound = np.array([
        [0, 500], # tresh
        [0, 20], # coef
        [0, 2000], # to_delete
        [0, 1000], # to_create
        [0, 20], # shape
        [0, 1000], # shift
        [0, 50], # ver_up
        [0, 300], # ver_down
        [0, 50], # hor_up
        [0, 150], # hor_down
        [0, 10000] #over
    ])
    
    count = 300
    
    # массив случайных чисел, но каждый столбец должен быть из своего диапазона
    arr = np.zeros((count, len(varbound)))
    
    for i, bound in enumerate(varbound):
        arr[:, i] = np.random.uniform(low = bound[0], high = bound[1], size = count)
    
    
    bees = Bees(arr, width = 0.2)
    
    hive = Hive(bees, 
            f, 
            parallel = False, # use parallel evaluating of functions values for each bee? (recommented for heavy functions, f. e. integtals) 
            verbose = True) # show info about hive 
    
    best_result, best_position = hive.get_result(max_step_count = 100, # maximun count of iteraions
                      max_fall_count = 30, # maximum count of continious iterations without better result
                      w = 0.3, fp = 2, fg = 5, # parameters of algorithm
                      latency = 1e-9, # if new_result/old_result > 1-latency then it was the iteration without better result
                      verbose = True # show the progress
                      )
    
    return best_position
    

In [ ]:
def evaluate_global(solution_seeker = find_solution, cv = 5, repeats = 5):
    """
    это функция для повторной кросс-валидации, но мы ее не использовали, потому что не было на это времени
    """
    
    total = []
    
    for _ in range(repeats):
    
        inds = np.random.randint(cv, size = target_frame.shape[0])

        for i in range(cv):
            mask = inds != i
            solution = solution_seeker(target_frame.iloc[mask,:])
            total.append(evaluate(target_frame.iloc[np.logical_not(mask),:], solution))

    return solution, np.mean(total)


# получить лучшие параметры в виде одномерного массива
best_params = find_solution(target_frame)

# Визуализация решения

По найденным параметрам строим решение (пока что в метриках штрафов):

In [ ]:
params = best_params

pr = {
        'to_delete': params[2],
        'to_create': params[3],
        'shape': params[4],
        'shift': params[5],
        'ver_up': params[6],
        'ver_down': params[7],
        'hor_up': params[8],
        'hor_down': params[9]
}
    
def some_function(sample_arr, expert_arr, name_sample, name_exp):
    return result_function(sample_arr, expert_arr, name_sample, name_exp, tresh = params[0], params = pr, coef = params[1], over = params[10])



answer = [some_function(samples_dict[sample_arr], expert_dict[remove_end(sample_arr)], sample_arr, remove_end(sample_arr)) for sample_arr in target_frame['names']]

res = target_frame.copy()
res['predicted'] = answer

res

Нарисуем полученные величины так, чтоб было видно, насколько удалось сохранить порядок:

In [ ]:
res = res.sort_values(['predicted'])

scatter = plt.scatter(np.arange(res.shape[0]), res['predicted'], c = res['target'].values)

plt.legend(handles = scatter.legend_elements()[0], labels = set(list(res['target'].values)))
plt.show()


Теперь нужно свести исходные диапазоны к требуемым (от 1 до 5). Создадим линейную модель, где исходный диапазон будет предиктором. Сейчас корреляция между целью и предиктором равна:

In [ ]:
stats.pearsonr(res['predicted'], res['target'])[0] 

Если сделать преобразование Бокса-Кокса, она вырастет почти в два раза:

In [ ]:
bx, lambd = stats.boxcox(res['predicted'])
print(stats.pearsonr(bx, res['target'])[0] )

In [ ]:
def tobx(vector):
    """
    делает то же самое преобразование, используя сохраненный параметр
    """
    if lambd == 0:
        return np.log(vector)
    return (vector**lambd - 1)/lambd

Визуализация после преобразования: 

In [ ]:
scatter = plt.scatter(np.arange(res.shape[0]), bx, c = res['target'].values)

plt.legend(handles = scatter.legend_elements()[0], labels = set(list(res['target'].values)))
plt.show()

Построим простейшую линейную модель по этим данным:

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(bx, res['target'])

slope, intercept, r_value, p_value, std_err   

Зафиксируем полученные преобразования в рамках одной функции:

In [ ]:
def convert2onefive(vector):
    answer = vector.copy()
    
    zero = vector == 0
    not_zero = np.logical_not(zero)
    
    vector2 = vector[not_zero]
    answer[not_zero] = tobx(vector[not_zero])*slope + intercept
    answer[zero] = 5 # 0 штраф -- считаем за лучший счет, значит класс 5 (исключения потом наложатся поверх, так что не страшно пока оставить этот код)
    
    answer[answer<1] = 1
    answer[answer>5] = 5
    
    return answer
    

Выполним предсказание на исходных данных:

In [ ]:
results = pd.DataFrame({
    'predicted': convert2onefive(res['predicted'].values),
    'goal': res['target']
})

print(f"MAE = {np.mean(np.abs(results['predicted'] - results['goal']))}")

results

Осталось сделать то же самое для тестовых данных.

# Предсказание

Прочтем образец:

In [ ]:
preds = pd.read_csv('../input/digital-transformation-2020/SecretPart_dummy.csv')

preds

Как и раньше, сведём его к таблице с двумя столбцами:

In [ ]:
preds2 = table_preparation(preds)

preds2

Собственно, сделаем само предсказание:

In [ ]:
answer = [some_function(samples_dict[sample_arr], expert_dict[remove_end(sample_arr)], sample_arr, remove_end(sample_arr)) for sample_arr in preds2['names']]

preds2['target'] = convert2onefive(np.array(answer))

# работа с "лишними" парами
preds2['target'][np.array([(len(samples_dict[sample_arr]) == 0) and (len(expert_dict[remove_end(sample_arr)]) == 0) for sample_arr in preds2['names']])] = 5 # если нет объектов в обоих снимках, это 5

preds2['target'][np.logical_not(np.array([(len(samples_dict[sample_arr]) > 0) and (len(expert_dict[remove_end(sample_arr)]) > 0) for sample_arr in preds2['names']]))] = 1 # если нет объектов только на одном из снимков, это 1
                          
preds2

Вставим полученные значения в исходную таблицу:

In [ ]:
for col, arr in zip(preds.columns[1:], np.array_split(preds2['target'].values, 3)):
    preds[col] = arr

preds

Сохраним решение:

In [ ]:
preds.to_csv('AK_output.csv', index = False)